In [437]:
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics,preprocessing
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV

import scipy
path = os.getcwd()
path='/Users/xtarx/Documents/TUM/4th/MLMA/ex1/SAheartdata/'

In [71]:
def read_file(file_name,y_index):
    data = pd.read_csv(path+file_name)
    y=data[data.columns[y_index]].as_matrix()
    x=data.drop(data.columns[[y_index]], axis=1).as_matrix()
    return  x,y

In [81]:
data = pd.read_csv(path + "SAheart.csv")
data.famhist=pd.Categorical(data.famhist).codes

In [117]:
data.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1


In [421]:
train_x = data.drop(data.columns[[-1]], axis=1).as_matrix()
train_y = data[data.columns[-1]].as_matrix()

In [422]:
train_x[:,0]

array([ 160.,  144.,  118.,  170.,  134.,  132.,  142.,  114.,  114.,
        132.,  206.,  134.,  118.,  132.,  112.,  117.,  120.,  146.,
        158.,  124.,  106.,  132.,  150.,  138.,  142.,  124.,  118.,
        145.,  144.,  146.,  136.,  158.,  122.,  126.,  148.,  122.,
        140.,  110.,  130.,  136.,  118.,  144.,  120.,  130.,  114.,
        128.,  162.,  116.,  114.,  126.,  122.,  134.,  152.,  134.,
        156.,  152.,  118.,  126.,  103.,  121.,  142.,  138.,  152.,
        140.,  130.,  136.,  124.,  112.,  118.,  122.,  118.,  130.,
        130.,  126.,  128.,  136.,  134.,  140.,  168.,  108.,  114.,
        140.,  148.,  148.,  128.,  130.,  126.,  140.,  126.,  122.,
        116.,  120.,  143.,  118.,  194.,  134.,  138.,  136.,  122.,
        164.,  136.,  166.,  118.,  128.,  118.,  158.,  108.,  170.,
        118.,  124.,  114.,  168.,  134.,  174.,  116.,  128.,  140.,
        154.,  150.,  130.,  128.,  120.,  120.,  138.,  153.,  123.,
        148.,  136.,

a) Create a model that contains only the intercept (null model), i.e. no features
are considered.

In [167]:
def null_model(y):
    logistic = LogisticRegression(C=1e6, solver='newton-cg', intercept_scaling=1e3)
    X = np.zeros(y.size).reshape(-1, 1)
    logistic.fit(X, y)
    pred = logistic.predict(X)
    ll = metrics.log_loss(y, pred)
    return pred, ll

Create multiple models each considering a single feature. Note that famhist
is a categorical feature which has to be converted to numbers first.

In [187]:
def full_model(X, y):
    logistic = LogisticRegression()
    # X=X.reshape(-1,1)
    logistic.fit(X, y)
    pred = logistic.predict(X)
    ll = metrics.log_loss(y, pred)
    return pred, ll

def model_wth_stats(X, y):
    logistic = LogisticRegression()
    X=X.reshape(-1,1)
    logistic.fit(X, y)
    pred = logistic.predict(X)
    ll = metrics.log_loss(y, pred)
    return pred, ll

Create a function likelihood_ratio_test implementing the likelihood-ratio
test which takes the log-likelihood of the full model and the reduced model
(Section 1.1). Use this function to compare the single feature models to the
null model. Which feature yields the most significant improvement over the
null model? Make sure you consider the p-value of the likelihood-ratio test.

In [196]:
def likelihood_ratio_test(full_model_ll, reduced_model_ll, df):
    D = -2 * (full_model_ll - reduced_model_ll)
    p = scipy.special.gammaincc(df / 2, D / 2)
    return p

In [183]:
null_pred, null_ll = null_model(train_y)
full_pred, full_ll = full_model(train_x, train_y)

In [145]:
sbp_pred, sbp_ll = model_wth_stats(train_x[:,4], train_y)

In [146]:
sbp_ll

11.961481002566471

In [225]:
models = []
models_ll = []
p_value = []

#Null model


for i in range(0, train_x.shape[1]):
    pred,ll=model_wth_stats(train_x[:,i],train_y)
    models.append(pred)
    models_ll.append(ll)
    p_value.append(likelihood_ratio_test(full_ll, ll, 1))

In [227]:
p_value

[0.022162154485560338,
 0.13424775557170157,
 0.037315655125469561,
 0.034184515102020641,
 0.018671475844365633,
 0.018671475844365633,
 0.018671475844365633,
 0.017144204216065061,
 0.037313151396706302]

In [228]:
models_ll

[11.8119728744166,
 10.316810248590579,
 11.363441567045514,
 11.438193900390081,
 11.961481002566471,
 11.961481002566471,
 11.961481002566471,
 12.036243720293246,
 11.36349868114767]

In [230]:
p_value

[0.022162154485560338,
 0.13424775557170157,
 0.037315655125469561,
 0.034184515102020641,
 0.018671475844365633,
 0.018671475844365633,
 0.018671475844365633,
 0.017144204216065061,
 0.037313151396706302]

d) Create a model which considers multiple features by starting with the null
model and adding one additional feature at a time. To determine which feature
to add, use the p-value as returned by the likelihood-ratio test. Extended
models with one additional feature, where the p-value is greater than 0.05,
should not be considered. In each step choose the model with the smallest
p-value. Continue until all features have been selected or the model cannot be
improved significantly any more. Print all selected features.

In [379]:
p_sorted_index = np.argsort(p_value)
#create null model
current_p = likelihood_ratio_test(full_ll, null_ll, 1)
# feature_new = np.zeros(train_y.size)
first_time=0
print(current_p)
feature_new = np.zeros(train_y.size)
for i in range(0, len(p_sorted_index)):

    if (p_value[p_sorted_index[i]] > 0.05):
        print("feature # ", p_sorted_index[i], " shall not be considerd as p is >0.05  --- ",
              p_value[p_sorted_index[i]])
        
    else:
        col_new = train_x[:, p_sorted_index[i]]
        
        feature_new = np.column_stack((feature_new, col_new))
        if(first_time==0):
            feature_new=feature_new[:,1:]
            first_time=99

        #we stacked the columns, now we calcuate the new logloss 
        _,ll_new=full_model(feature_new,train_y)
        print("ll new is ", ll_new)
        print("feature #", p_sorted_index[i], " was added to our list")
        p_new=likelihood_ratio_test(full_ll,ll_new,1)
        # if (p_new>= current_p ):
        #     break

0.0186714758444
ll new is  12.0362437203
feature # 7  was added to our list
ll new is  11.7372413098
feature # 4  was added to our list
ll new is  11.5129860405
feature # 5  was added to our list
ll new is  11.2886996181
feature # 6  was added to our list
ll new is  10.3915885429
feature # 0  was added to our list
ll new is  9.34496760882
feature # 3  was added to our list
ll new is  9.64402367192
feature # 8  was added to our list
ll new is  9.41974244166
feature # 2  was added to our list
feature #  1  shall not be considerd as p is >0.05  ---  0.134247755572


In [380]:
_, ll_new = full_model(feature_new, train_y)
print("ll new is ", ll_new)

ll new is  9.41974244166


L1 (lasso) regularization can also be used for feature selection. Consider a
full model with all features as input, penalized with the L1 norm of coefficients
(try regularization parameter C in the range of 0.01 − 0.1). Features
with an non-zero coefficient are important for the classification. Compare the
Lasso-selected features to the features selected by p-values. Please note Lassofeature
selection requires a standardization of features that each feature has a
zero mean and a unit standard derivation (e.g.using Sklearn built-in function
sklearn.preprocessing.scale)

In [500]:
def lasso(X, y):
    # X = preprocessing.scale(X)
    # clf = LassoCV(alphas = [ 0.1, 0.001, 0.0005])
    # sfm = SelectFromModel(clf)
    # sfm.fit(X, y)
    # n_features = sfm.transform(X).shape[1]
    # return sfm
    lsvc = LassoCV(alphas = [ 0.1, 0.001, 0.0005]).fit(X, y)
    model = SelectFromModel(lsvc, prefit=True)
    X_new = model.transform(X)
    return X_new.shape

In [501]:
lassofeatures=lasso(train_x, train_y)

In [503]:
lassofeatures

(462, 9)